In [1]:
import torch
from torch.utils.tensorboard import SummaryWriter

import datamaestro
from tqdm import tqdm
import numpy as np

In [2]:
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.nn.functional import linear
from torch.nn import MSELoss

In [3]:
from tp1 import MSE, linear, Context

## Normalize boston Data

In [4]:
data=datamaestro.prepare_dataset("edu.uci.boston")
colnames, datax, datay = data.data()
datax = (datax-datax.min(axis=0))/(datax.max(axis=0)-datax.min(axis=0))

## Apply TME1 functions

In [5]:
#writer = SummaryWriter()

# Les données supervisées
x = torch.tensor(datax).float()
y = torch.tensor(datay.reshape((len(datay), 1))).float()

# Les paramètres du modèle à optimiser
w = torch.randn(13, 1)
b = torch.randn(1)

epsilon = 0.001

writer = SummaryWriter()
for n_iter in range(1000):
    # Pass Forward
    ctx1 = Context()
    ctx2 = Context()

    yhat = linear.forward(ctx1,x,w,b)
    loss = MSE.forward(ctx2,yhat,y)

    # Back propagation
    grad_yhat, grad_y = MSE.backward(ctx2, 1)
    grad_x, grad_w, grad_b = linear.backward(ctx1, grad_yhat)

    #import ipdb;ipdb.set_trace()

    # Tensorboard visualization
    # To visualize type command : tensorboard --logdir=runs in current directory
    writer.add_scalar('Loss/train', loss, n_iter)
    if(n_iter%100==0):
        print(f"Itérations {n_iter}: loss {loss}")

    # Updating parameters
    w -= epsilon*grad_w
    b -= epsilon*grad_b

#writer.close()

Itérations 0: loss 596.6293334960938
Itérations 100: loss 222.6676483154297
Itérations 200: loss 134.39857482910156
Itérations 300: loss 109.70768737792969
Itérations 400: loss 99.64109802246094
Itérations 500: loss 93.34754180908203
Itérations 600: loss 88.36827850341797
Itérations 700: loss 84.10001373291016
Itérations 800: loss 80.35552978515625
Itérations 900: loss 77.04669952392578


## Apply with torch functions

In [6]:

# Les données supervisées
x = torch.tensor(datax).float()
y = torch.tensor(datay.reshape((len(datay), 1))).float()

# Les paramètres du modèle à optimiser
w =  torch.nn.Parameter(torch.randn((1, 13)))
b =  torch.nn.Parameter(torch.randn(1))

epsilon = 0.001

optim = torch.optim.SGD(params=[w,b],lr=epsilon)
optim.zero_grad()

mse = MSELoss()

size_batch = len(x)

#writer = SummaryWriter()
for n_iter in range(1000):
    
    indices = torch.randperm(len(x))
    for i in range(int(np.ceil((len(x)/size_batch)))):
        # Pass Forward
        index = indices[i*size_batch:(i+1)*size_batch]
        x_batch = x[index]
        y_batch = y[index]
        yhat = torch.nn.functional.linear(x_batch,w,b)
        loss = mse(yhat,y_batch)

        loss.backward()

        #import ipdb;ipdb.set_trace()

        # Tensorboard visualization
        # To visualize type command : tensorboard --logdir=runs in current directory
        #writer.add_scalar('Loss/train', loss, n_iter)
        
        # Updating parameters & reset grad
        optim.step()
        optim.zero_grad()
        
    if(n_iter%100==0):
        print(f"Itérations {n_iter}: loss {loss}")

#writer.close()

Itérations 0: loss 581.64599609375
Itérations 100: loss 228.9104461669922
Itérations 200: loss 144.177001953125
Itérations 300: loss 119.27262115478516
Itérations 400: loss 108.29158020019531
Itérations 500: loss 101.0372314453125
Itérations 600: loss 95.18082427978516
Itérations 700: loss 90.13550567626953
Itérations 800: loss 85.70738220214844
Itérations 900: loss 81.79731750488281


## Try 2 couches (linear->tanh->linear->MSE)

In [7]:

# Les données supervisées
x = torch.tensor(datax).float()
y = torch.tensor(datay.reshape((len(datay), 1))).float()

# Les paramètres du modèle à optimiser
w1 =  torch.nn.Parameter(torch.randn((1, 13)))
b1 =  torch.nn.Parameter(torch.randn(1))

w2 =  torch.nn.Parameter(torch.randn((1,1)))
b2 =  torch.nn.Parameter(torch.randn(1))

epsilon = 0.001

optim = torch.optim.SGD(params=[w1,b1,w2,b2],lr=epsilon)
optim.zero_grad()

mse = MSELoss()

size_batch = len(x)

#writer = SummaryWriter()
for n_iter in range(1000):
    
    indices = torch.randperm(len(x))
    for i in range(int(np.ceil((len(x)/size_batch)))):
        #Compute sample
        index = indices[i*size_batch:(i+1)*size_batch]
        x_batch = x[index]
        y_batch = y[index]
        
        # Pass Forward
        out = torch.nn.functional.linear(x_batch,w1,b1)
        out = torch.nn.functional.tanh(out)
        yhat = torch.nn.functional.linear(out,w2,b2)
        loss = mse(yhat,y_batch)
        
        # Pass backward
        loss.backward()
        
        # Updating parameters & reset grad
        optim.step()
        optim.zero_grad()
        
    if(n_iter%100==0):
        print(f"Itérations {n_iter}: loss {loss}")

#writer.close()

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Itérations 0: loss 561.5813598632812
Itérations 100: loss 300.8995361328125
Itérations 200: loss 181.7779541015625
Itérations 300: loss 128.1973114013672
Itérations 400: loss 104.13251495361328
Itérations 500: loss 93.32368469238281
Itérations 600: loss 88.46626281738281
Itérations 700: loss 86.28099060058594
Itérations 800: loss 85.29585266113281
Itérations 900: loss 84.85005187988281


## Use Module

In [8]:
# Les données supervisées
x = torch.tensor(datax).float()
y = torch.tensor(datay.reshape((len(datay), 1))).float()

# Module
model = torch.nn.Sequential(
          torch.nn.Linear(13, 1, bias=True),
          torch.nn.Tanh(),
          torch.nn.Linear(1, 1, bias=True)
        )

epsilon = 0.001

optim = torch.optim.SGD(params=model.parameters(),lr=epsilon)
optim.zero_grad()

criterion = MSELoss()

size_batch = len(x)

#writer = SummaryWriter()
for n_iter in range(1000):
    
    indices = torch.randperm(len(x))
    for i in range(int(np.ceil((len(x)/size_batch)))):
        #Compute sample
        index = indices[i*size_batch:(i+1)*size_batch]
        x_batch = x[index]
        y_batch = y[index]
        
        # Pass Forward
        yhat = model(x_batch)
        loss = criterion(yhat, y_batch)
        
        # Pass backward
        loss.backward()
        
        # Updating parameters & reset grad
        optim.step()
        optim.zero_grad()
        
    if(n_iter%100==0):
        print(f"Itérations {n_iter}: loss {loss}")

#writer.close()

Itérations 0: loss 598.0997924804688
Itérations 100: loss 307.7272644042969
Itérations 200: loss 184.63973999023438
Itérations 300: loss 129.40939331054688
Itérations 400: loss 104.62705993652344
Itérations 500: loss 93.50627899169922
Itérations 600: loss 88.51509094238281
Itérations 700: loss 86.2741928100586
Itérations 800: loss 85.26737213134766
Itérations 900: loss 84.81431579589844
